In [204]:
"""
Created on Thu Sep  16 14:53:08 2019

@author: marcos
"""

import pandas as pd
import numpy as np
import scipy.stats as stats
import heapq 
import pickle
%matplotlib inline
    
import plotnine as p9
import matplotlib.pyplot as plt
import dateutil
import datetime 
from time import time
import re
from scipy import interpolate
import timeit
from dateutil.rrule import DAILY, rrule, MO, TU, WE, TH, FR
import seaborn as sns

from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression, Lasso, BayesianRidge
from sklearn import tree
import math

import sys
sys.path.append("../")


from historical.readData.old.estimateData3 import read

from historical.readData.old.estimateData3 import read
from historical.readData.old.estimateData3 import search_travels
from historical.readData.old.estimateData3 import estimate
from historical.readData.old.estimateData3 import stops_distance

from historical.readData.old.travels3 import haversine2

def calcula_dist_shape(selec_linhas):
    distance_all_shapes = {}
    # print 'Calculando distancias dos shapes'
    '''calcula todas as distancias dos shapes'''
    
    pth_files_GTFS = "../data/gtfs/"    
        
    trips = pd.read_csv(pth_files_GTFS + 'trips.txt', sep=',')
    shapes = pd.read_csv(pth_files_GTFS + 'shapes.txt', sep=',')
    
    for l, trip_id in enumerate(selec_linhas):

        trip = trips[trips.trip_id == trip_id]
        trip_shape = shapes[shapes['shape_id'].isin(trip['shape_id'])]
        shapelat = trip_shape.shape_pt_lat.tolist()
        shapelon = trip_shape.shape_pt_lon.tolist()

        # distancias dos shapes
        lon1 = shapelon[0]
        lat1 = shapelat[0]
        totalcal = [0.]
        dist = [0.]
        for lat2, lon2 in zip(shapelat[1:], shapelon[1:]):
            d = haversine2(lat1, lon1, lat2, lon2)
            d = d * 1000
            dist.append(d)
            totalcal.append(totalcal[-1] + d)
            lat1 = lat2
            lon1 = lon2
        distance_all_shapes[trip_id] = [shapelat, shapelon, totalcal]
    return distance_all_shapes

def stops_distance(linha):

    distances = calcula_dist_shape([linha])
    totalcal = distances[linha][2]
    
    pth_files_GTFS = "../data/gtfs/"
    

    trips = pd.read_csv(pth_files_GTFS + 'trips.txt', sep=',')
    shapes = pd.read_csv(pth_files_GTFS + 'shapes.txt', sep=',')
    stops = pd.read_csv(pth_files_GTFS + 'stops.txt', sep=',')
    stopid = pd.read_csv(pth_files_GTFS + 'stop_times.txt', sep=',')   

    ida = trips[trips.trip_id == linha]
    shapeida = shapes[shapes['shape_id'].isin(ida['shape_id'])]
    idalat = shapeida.shape_pt_lat.tolist()
    idalon = shapeida.shape_pt_lon.tolist()
    total = shapeida.shape_dist_traveled.tolist()

    temp1 = stopid[stopid.trip_id == linha]
    stopsida = stops[stops['stop_id'].isin(temp1['stop_id'])]
    stopsida = stopsida.set_index('stop_id')
    stopsida = stopsida.reindex(index=temp1['stop_id'])

    stopslat = stopsida.stop_lat.tolist()
    stopslon = stopsida.stop_lon.tolist()

    dpontos = [None] * len(stopslat)
    index = 0
    lat = idalat
    lon = idalon
    total = 0
    p = ['depois'] * len(stopslat)
    for latb, lonb, i in zip(stopslat, stopslon, range(len(stopslat))):
        lat = lat[index:]
        lon = lon[index:]
        nn = haversine2(latb, lonb, np.array(lat), np.array(lon)) * 1000
        index = nn.argmin()
        total = total + index
        if index == 0:
            dpontos[i] = nn[index]
#            descontardist = nn[index]
        else:
            if totalcal[total] >= totalcal[total - 1] + nn[index - 1]:
                p[i] = 'antes'
            dpontos[i] = totalcal[total - 1] + nn[index - 1]

    mid_points = []
#    mid_points.append(200)
    p1 = dpontos[0]
    for p2 in dpontos[1:]:
        mid_points.append(p1 + (p2-p1)/2)
        p1 = p2
    dpontos = [i/1000. for i in dpontos]
    mid_points = [i/1000. for i in mid_points]

    return dpontos, mid_points

def read(filedata,filerep):
    df0 = pd.read_pickle(filedata, compression=None)
    with open(filerep, 'rb') as handle:
        reps = pickle.load(handle, encoding='latin1')
    return df0, reps

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + datetime.timedelta(n)
        
def daterangeWD(start_date, end_date):
  return rrule(DAILY, dtstart=start_date, until=end_date, byweekday=(MO,TU,WE,TH,FR))

def hr_func(ts):
    return ts.hour

def minute_func(ts):
    return ts.minute

def second_func(ts):
    return ts.second


def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

import warnings
warnings.filterwarnings('ignore')

%matplotlib agg 

pth_files_GTFS = "../data/gtfs/"

trips = pd.read_csv(pth_files_GTFS + 'trips.txt', sep=',')
shapes = pd.read_csv(pth_files_GTFS + 'shapes.txt', sep=',')
stops = pd.read_csv(pth_files_GTFS + 'stops.txt', sep=',')
stopid = pd.read_csv(pth_files_GTFS + 'stop_times.txt', sep=',')   

periods = ['morning', 'm_peak', 'i_peak', 'a_peak', 'night']

selected_line = ['8700-10-1']

selected_line = [
                '8700-10-1',                
                '7545-10-1',
                '7545-10-0',
                '6450-10-1',
                '6450-10-0',
                '3301-10-1',
#                 '3301-10-0',
                '2290-10-1',
                '2290-10-0',
#                 '574J-10-1',
#                 '574J-10-0',
#                 '477P-10-1',
                '477P-10-0',
#                 '351F-10-1',
#                 '351F-10-0'
                ]

# selected_line = ['6450-10-0']
selected_line = [ '6450-10-0', '8700-10-1']


In [205]:
Headway = []


for line in selected_line:

    p, mp = stops_distance(line)
    pth_files_lines = "../historical/readData/"

    df, reps = read(pth_files_lines + "trips_" + line + ".dsk", pth_files_lines + "interps_" + line + ".rep")

    df['day'] = pd.to_datetime(df['day'], format = '%Y-%m-%d')
    df['start'] = pd.to_datetime(df['start'], format = '%H:%M:%S')

    start_date = datetime.date(2017, 9, 1)       
    end_date = datetime.date(2017, 9, 28)        

    df = df.loc[((df['day'] >= start_date) & (df['holiday'] != 1)) & ((df['weekday'] != 6) & (df['weekday'] != 5) & (df['weekday'] != 4) & (df['weekday'] != 0))]

    df['exact_time'] = np.array(df['day'], dtype='datetime64[ns]') + \
                        pd.to_timedelta(pd.to_timedelta(pd.DatetimeIndex(df['start']).hour*60 + \
                        pd.DatetimeIndex(df['start']).minute + \
                        pd.DatetimeIndex(df['start']).second/60 + \
                        df['time'], unit='m'))

    df.index = df['exact_time']
    df['day_hour'] = df['exact_time'].apply(hr_func)

    df['link'] = 0
    for i in range(0, len(mp)):
        if (i == 0):
              df.loc[(df['distance'] > 0) & (df['distance'] < mp[i+1]),'link'] = i 
        if (i == len(mp) -1):
            df.loc[(df['distance'] > mp[i]) & (df['distance'] < max(p)),'link'] = i
        if (i != 0) & (i != len(mp)-1):
            df.loc[(df['distance'] >= mp[i]) & (df['distance'] < mp[i+1]),'link'] = i
            
    df = df.drop_duplicates(subset=['day', 'numtravel', 'link', 'day_hour'])

    link_df = []
    travels = sorted(list(set(df.numtravel.unique())))
    for tr in travels:
        tck = reps[tr][0]
        tck_mods = [(tck[0],tck[1]-m,tck[2]) for m in mp]       
        tempo = [a[0] if a.size>0 else np.nan for a in [interpolate.sproot(tck_mod) for tck_mod in tck_mods]]
        row = [tuple([tempo[i+1]-pos for i, pos in enumerate(tempo[:-1])])]    
        link_df.append(row)        
        
    # data_training['time_link'] = 0 
    link_df_np = np.array(link_df)
    link_df_np.shape = (np.shape(link_df_np)[0] * np.shape(link_df_np)[2])

    # df['time_link'] = 0

    right = pd.DataFrame({'numtravel': np.repeat(df.numtravel.unique(), len(mp)-1), \
                  'link': np.tile(range(0, len(mp)-1), len(df.numtravel.unique())), 
                  'time_link': link_df_np}) 

    df_merged = pd.merge(right, df, how='right', on=['numtravel', 'link'])    
    
    df_merged.index = df_merged['exact_time']

    travel_times = pd.DataFrame(np.reshape(link_df, (np.shape(link_df)[0], np.shape(link_df)[2])))
    travel_times.index = travels

    for select_day in df.loc[df['day'] >= start_date].day.dt.strftime('%Y-%m-%d').unique():
        
        for link in range(1, len(mp) - 3):

            df_temp = df.loc[(df['day'] == select_day) & (df['link'] == link)]
            # travels_day = np.unique(np.array(df_temp['numtravel']))

            for hour_day in range(480, 1260, 60):   
                int_part, float_part = divmod((hour_day/60), 1)
                hour_begin = str(int(int_part)) + ':' + '00'
                hour_end = str(int(int_part)) + ':' + '59'
                hour_after = str(int(int_part) + 1) + ':' + '59'

                df_temp_time = df_temp.iloc[df_temp.index.indexer_between_time( hour_begin, hour_end)]
                temp_travels_time = travel_times.loc[np.unique(np.array(df_temp_time['numtravel']))][link]
                
                df_temp_after = df_temp.iloc[df_temp.index.indexer_between_time( hour_end, hour_after)]
                temp_travels_after = travel_times.loc[np.unique(np.array(df_temp_after['numtravel']))][link]                
                
                temp_travels_time = travel_times.loc[np.unique(np.array(df_temp_time['numtravel']))][link]
                time_after       = travel_times.loc[np.unique(np.array(df_temp_after['numtravel']))][link]                
                
                median_test = np.median(temp_travels_time)
                
                
                if ((len(df_temp_time) == 1) & (len(df_temp_after) > 0)):
                    Headway.append([(df_temp_after.sort_index().iloc[0]['exact_time'] - 
                                df_temp_time.sort_index().iloc[0]['exact_time']).seconds/60,
                                travel_times.loc[np.unique(np.array(df_temp_time['numtravel']))[0]][link],
                                line, link, select_day, hour_begin + ' - ' + hour_end
                                ])
                    
                elif ((len(df_temp_time) > 1) & (len(df_temp_after) > 0)):                    
                    for values_h in range(len(df_temp_time) - 1):
                        Headway.append([(df_temp_time.sort_index().iloc[values_h + 1]['exact_time'] - 
                                df_temp_time.sort_index().iloc[values_h]['exact_time']).seconds/60, 
                                travel_times.loc[np.unique(np.array(df_temp_time['numtravel']))[values_h]][link],
                                line, link, select_day, hour_begin + ' - ' + hour_end])
                                      
                        
                    Headway.append([(df_temp_after.sort_index().iloc[0]['exact_time'] - 
                                df_temp_time.sort_index().iloc[values_h + 1]['exact_time']).seconds/60,
                                travel_times.loc[np.unique(np.array(df_temp_time['numtravel']))[values_h + 1]][link],
                                line, link, select_day, hour_begin + ' - ' + hour_end
                                ])
    
    print('Ready line: ' + line )                
Headway = pd.DataFrame(Headway,columns=['headway', 'time_link', 'line', 'link', 'day', 'time'])

Ready line: 6450-10-0
Ready line: 8700-10-1


In [207]:
Headway_corr = []
for line in selected_line:
    for select_day in df.loc[df['day'] >= start_date].day.dt.strftime('%Y-%m-%d').unique():      
        for link in range(1, len(mp) - 3):
            for hour_day in range(480, 1260, 60):   
                int_part, float_part = divmod((hour_day/60), 1)
                hour_begin = str(int(int_part)) + ':' + '00'
                hour_end = str(int(int_part)) + ':' + '59'
                hour_after = str(int(int_part) + 1) + ':' + '59'
                
#                 hour_begin + ' - ' + hour_end
                
                var_temp = Headway.loc[(Headway['time'] == hour_begin + ' - ' + hour_end) & 
                                      (Headway['link'] == link) &  (Headway['line'] == line)
                                     ]
        
                Headway_corr.append([line, link, hour_begin + ' - ' + hour_end, 
                              var_temp['headway'].corr(var_temp['time_link'])])

In [210]:
Headway.loc[(Headway['time'] == hour_begin + ' - ' + hour_end) & 
                                      (Headway['link'] == link) &  (Headway['line'] == line)]

,headway,time_link,line,link,day,time
20192,0.416667,1.761585,8700-10-1,33,2017-09-05,20:00 - 20:59
20193,1.450000,1.184613,8700-10-1,33,2017-09-05,20:00 - 20:59
20194,4.583333,1.460540,8700-10-1,33,2017-09-05,20:00 - 20:59
20195,1.383333,1.244701,8700-10-1,33,2017-09-05,20:00 - 20:59
20196,0.583333,1.322357,8700-10-1,33,2017-09-05,20:00 - 20:59
20197,5.583333,1.396174,8700-10-1,33,2017-09-05,20:00 - 20:59
20198,7.816667,1.606390,8700-10-1,33,2017-09-05,20:00 - 20:59
20199,0.800000,2.334570,8700-10-1,33,2017-09-05,20:00 - 20:59
20200,3.916667,1.660137,8700-10-1,33,2017-09-05,20:00 - 20:59
20201,8.083333,1.164951,8700-10-1,33,2017-09-05,20:00 - 20:59


In [199]:
data = Headway_corr
        
fig = (
        p9.ggplot(data, p9.aes(x = 'link', y ='corr_headway')) +
        p9.geom_line() +
        p9.theme_bw() +
        p9.theme(figure_size = (8, 12)) +
        p9.facet_wrap('~line + hour_day') +
        p9.labs(title = 'Correlation Headway and link states of the lines') +
        p9.theme(figure_size = (9, 12))  
#         p9.theme(axis_text_x = p9.element_text(angle=90)) 
    )
p9.ggsave(fig, 'png/Headway_link_state_60M_corr' + '.png')  
# p9.ggsave(fig, 'pdf/data-Headway60M_median_' + '_' + select_day + '_30M_median.pdf')


In [200]:
data = Headway_corr       

fig = (
        p9.ggplot(data, p9.aes(x = 'factor(hour_day)', y ='corr_headway')) +
        p9.geom_bar(stat='identity') +
        p9.theme_bw() +
        p9.theme(figure_size = (8, 12)) +
        p9.facet_wrap('~line + link') +
        p9.labs(title = 'Correlation Headway and link states of the lines') +
        p9.theme(figure_size = (9, 12))  +
        p9.theme(axis_text_x = p9.element_blank()) 
    )
p9.ggsave(fig, 'png/Headway_60M_corr_line+link' + '.png')  
# p9.ggsave(fig, 'pdf/data-Headway60M_median_' + '_' + select_day + '_30M_median.pdf')


In [201]:
Headway_corr = []
for line in selected_line:
    for select_day in df.loc[df['day'] >= start_date].day.dt.strftime('%Y-%m-%d').unique():      
        for link in range(1, len(mp) - 3):

            df_temp = df.loc[(df['day'] == select_day) & (df['link'] == link)]
            # travels_day = np.unique(np.array(df_temp['numtravel']))

            for hour_day in range(480, 1260, 60):   
                int_part, float_part = divmod((hour_day/60), 1)
                hour_begin = str(int(int_part)) + ':' + '00'
                hour_end = str(int(int_part)) + ':' + '59'
                hour_after = str(int(int_part) + 1) + ':' + '59'
                
#                 hour_begin + ' - ' + hour_end
                
                df_temp = Headway.loc[(Headway['time'] == hour_begin + ' - ' + hour_end) & 
                                      (Headway['link'] == link) &  (Headway['line'] == line) &
                                      (Headway['day'] == select_day)
                                     ]
                
                if (len(df_temp) >= 2):                    
                    Headway_corr.append([select_day, line, link, hour_begin + ' - ' + hour_end, 
                              df_temp['headway'].corr(df_temp['time_link'])])

KeyboardInterrupt: 

In [ ]:
Headway_corr = pd.DataFrame(Headway_corr,columns=['line', 'link', 'hour_day', 'corr_headway', 'select_day'])


Headway_corr['hour_day'] = Headway_corr['hour_day'].astype('category')
Headway_corr['hour_day'] = Headway_corr['hour_day'].cat.reorder_categories(['8:00 - 8:59', '9:00 - 9:59', '10:00 - 10:59', 
                                                 '11:00 - 11:59', '12:00 - 12:59', '13:00 - 13:59', 
                                                 '14:00 - 14:59', '15:00 - 15:59',                                                  
                                                 '16:00 - 16:59', '17:00 - 17:59', '18:00 - 18:59', 
                                                 '19:00 - 19:59', '20:00 - 20:59'])

data = Headway_corr
        
fig = (
        p9.ggplot(data, p9.aes(x = 'link', y ='corr_headway')) +
        p9.geom_boxplot() +
        p9.theme_bw() +
        p9.theme(figure_size = (8, 12)) +
        p9.facet_wrap('~line + hour_day') +
        p9.labs(title = 'Correlation Headway and link states of the lines') +
        p9.theme(figure_size = (9, 12))  
#         p9.theme(axis_text_x = p9.element_text(angle=90)) 
    )
p9.ggsave(fig, 'png/Headway_link_state_60M_corr' + '.png')  
# p9.ggsave(fig, 'pdf/data-Headway60M_median_' + '_' + select_day + '_30M_median.pdf')
